In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# import math
import numpy as np
from typing import Any, Optional, List
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision

In [ ]:
torch.backends.cudnn.version()

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
# import numpy as np

In [ ]:
import torch.autograd.profiler as profiler

In [ ]:
import pytorch_lightning as pl

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (18,12) # set default size of plots

In [ ]:
import stylegan, data_module

In [ ]:
gan = stylegan.GanTask(gamma=28, ppl_reg_every=4, penalize_d_every=16, resolution=512)

In [ ]:
gan = gan.load_from_checkpoint(checkpoint_path="lightning_logs/version_15/checkpoints/epoch=8-step=57982.ckpt", strict=False)

In [ ]:
# dick = torch.load('new_synthesis_net.pt')
# gan.synthesis_net.load_state_dict(dick, strict=True)
# dick = torch.load('mapping_network.pt')
# gan.mapping_net.load_state_dict(dick, strict=True)


In [ ]:
dm = data_module.DataModule(data_dir='/mnt/sdd3/Limner/celeba_cartoons/',
                                batch_size=16, 
                                balanced_sampling=True,resolution=512)

In [ ]:
dm = data_module.DataModule(data_dir='./dataset',
                                batch_size=16, 
                                balanced_sampling=True,resolution=512)

In [ ]:
# dm.setup()

In [ ]:
# dl = dm.train_dataloader()

In [ ]:
# for batch in dl:
#     break

In [ ]:
# labels = F.one_hot(batch[1].long(), 2).float().cuda() #TODO: remove


In [ ]:
# gan = gan.cuda()

In [ ]:
# batch[0] = batch[0].cuda()

In [ ]:
# batch[1] = batch[1].cuda()

In [ ]:
# with profiler.profile(record_shapes=True, profile_memory=True, use_cuda=True, with_stack=True) as prof:
#     with profiler.record_function("model_inference"):
#         gan.training_step(batch, 0, 0)

In [ ]:
# with profiler.profile(record_shapes=True, profile_memory=True, use_cuda=True, with_stack=True) as prof:
#     with profiler.record_function("model_inference"):
#         gan(4, labels)

In [ ]:
# kav = prof.key_averages()

In [ ]:
# kav.total_average()

In [ ]:
# a = torch.randn(1000000, 3)
# b = torch.randn(1000000, 3)

# with torch.autograd.profiler.profile(use_cuda=True) as prof:
#     torch.mm(a.T, b)
# print(prof.key_averages().table(sort_by="self_cpu_time_total"))
# # Self CPU time total: 12.777ms
# # CUDA time total: 12.951ms

# a, b = a.cuda(), b.cuda()
# with torch.autograd.profiler.profile(use_cuda=True) as prof:
#     torch.mm(a.T, b)
# print(prof.key_averages().table(sort_by="self_cpu_time_total"))

In [ ]:
class Hooks(pl.Callback):
    num = 0
    one_hot = F.one_hot(
                    torch.cat(
                        (torch.zeros((3,)).long(),
                        torch.ones((3,)).long())
                    ), 2).float()
    def on_epoch_end(self, trainer, model: stylegan.GanTask):
        with torch.no_grad():
            fake_images, fake_latents = model.forward(6, self.one_hot.cuda())
            tensorboard = trainer.logger.experiment
            grid = torchvision.utils.make_grid(fake_images.cpu(), normalize=True, range=(-1, 1))
            tensorboard.add_image('images', grid, self.num)
            self.num += 1

In [ ]:
trainer = pl.Trainer(
    benchmark=True,
    automatic_optimization=False, gpus=1, 
    accumulate_grad_batches=1, 
    precision=32,
    callbacks=[Hooks()],
#   amp_backend='native', gradient_clip_val=10,
#   amp_level='01',
#   terminate_on_nan=True
)

In [ ]:
trainer.fit(gan, dm)

In [ ]:
gan = gan.cpu()

In [ ]:
# trainer.save_checkpoint("example_512_32.ckpt")
# new_model = MyModel.load_from_checkpoint(checkpoint_path="example.ckpt")


In [ ]:
fake_images.requires_grad

In [ ]:
# with torch.no_grad():
gan = gan.cuda()
fake_images, fake_latents = gan.forward(8, F.one_hot(torch.ones((8,)).long(), 2).float().cuda(), mix_prob=1)
# fake_latents.requires_grad_(True)
# fake_images.requires_grad_(True)
path_loss, path_lengths = gan.ppl_regularization(fake_images.cuda(), fake_latents.cuda())

In [ ]:
path_loss

In [ ]:
path_lengths

In [ ]:
# path_loss, path_lengths = gan.ppl_regularization(fake_images, fake_latents)

In [ ]:
# gan.requires_grad_(True)

In [ ]:
# fake_images, fake_latents = gan.forward(8, F.one_hot(torch.zeros((8,)).long(), 2).float(), mix_prob=0)
# path_loss, path_lengths = gan.ppl_regularization(fake_images, fake_latents)

In [ ]:
# path_lengths

In [ ]:
with torch.no_grad():
    fake_images, fake_latents = gan(8, F.one_hot(torch.zeros((8,)).long(), 2).float())

In [ ]:
path_lengths

In [ ]:
def make_image(tensor):
    return (
        tensor.detach()
        .clamp_(min=-1, max=1)
        .add(1)
        .div_(2)
        .mul(255)
        .type(torch.uint8)
        .permute(1, 2, 0)
        .to("cpu")
        .numpy())

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
noise = gan.synthesis_net.make_noise(1)

In [ ]:
@interact
def play_with_mix(i=(0,7,1), j=(0,7,1), c=(0,1,0.05), nc=(0,1,0.1)):
    with torch.no_grad():
    #     print(path_lengths[j].item())
        n = torch.randn_like(fake_latents[i][None, :, :]) * 0.2
        latend = (c * fake_latents[i][None, :, :] + (1 -c) * fake_latents[j][None, :, :]) 
        plt.imshow(make_image(gan.synthesis_net.forward(latend + n * nc, noise)[0]))

In [ ]:
fake_latents.shape

In [ ]:
# flatents = fake_latents[:,0]

In [ ]:
with torch.no_grad():
    fake_images, fake_latents = gan.forward(8, F.one_hot(torch.zeros((8,)).long(), 2).float(), mix_prob=1)

In [ ]:
flatents = torch.flatten(fake_latents,start_dim=1)

In [ ]:
# flatents_photo = torch.flatten(fake_latents_photo,start_dim=1)


In [ ]:
torch.cdist(flatents, flatents)[0] 

In [ ]:
with torch.no_grad():
    img = gan.synthesis_net(fake_latents)
    lblz = F.one_hot(torch.zeros((8,)).long(), 2).float()
    _, activations = gan.critic_net.forward(img, lblz, return_activations=True)
    projection = gan.auxiliary_projector.forward(activations[-1])

In [ ]:
torch.cdist(projection, projection)[0] 

In [ ]:
F.mse_loss(torch.cdist(flatents, flatents)[0] , torch.cdist(projection, projection)[0] )

In [ ]:
import scipy

In [ ]:
from scipy import spatial

In [ ]:
bath_size = 32
with torch.no_grad():
    latents = []
    for _ in range(128):
        z = torch.randn((bath_size, 512))
        l =  F.one_hot(torch.zeros((bath_size, )).long(), 2).float()
        w = gan.mapping_net.forward(z, l).detach().numpy()
        latents.append(w)

In [ ]:
bath_size = 32
with torch.no_grad():
    latents_photo = []
    for _ in range(128):
        z = torch.randn((bath_size, 512))
        l =  F.one_hot(torch.ones((bath_size, )).long(), 2).float()
        w = gan.mapping_net.forward(z, l).detach().numpy()
        latents_photo.append(w)

In [ ]:
def unfuck(x):
    return np.maximum(0, x) + 5.0 * np.minimum(0, x)

In [ ]:
latents_photo =  np.concatenate(latents_photo, axis=0)

In [ ]:
latents = np.concatenate(latents, axis=0)

In [ ]:
np.save('latents.npy', unfuck(latents))
np.save('latents_photo.npy', unfuck(latents_photo))

In [ ]:
for i in range(32):
    _ = plt.hist(unfuck(latents[:, i]), bins=100)
    _ = plt.hist(unfuck(latents_photo[:, i]), bins=100, alpha=0.5)
    plt.show()

In [ ]:
d = spatial.distance.cdist(latents, latents)

In [ ]:
d_id = spatial.distance.cdist(latents, latents_photo)

In [ ]:
d_p = spatial.distance.cdist(latents_photo, latents_photo)

In [ ]:
d_flat = d.ravel()[d.ravel() > 0]

In [ ]:
d_id_flat = d_id.ravel()[d_id.ravel() > 0]

In [ ]:
d_p = d_p.ravel()[d_p.ravel() > 0]

In [ ]:
plt.hist(d_flat, bins=100)
plt.hist(d_id_flat, bins=100, alpha=0.5)
plt.hist(d_p, bins=100, alpha=0.5, label='photos')

In [ ]:
r = torch.randperm(8)
r

In [ ]:
torch.pairwise_distance(a1_flat, a1_flat[r])

In [ ]:
with torch.no_grad():
    gan.critic_net(a[0], F.one_hot(torch.randint(0, 2, (4,)), 2).float())

In [ ]:
dl = dm.train_dataloader()

In [ ]:
for imz, lbz in dl:
    break

In [ ]:
lbz[1]

In [ ]:
a_cpu = make_image(imz[3])
plt.imshow(a_cpu)

37/40875 [03:16<60:23:08, 5.32s/it, loss=nan, v_num=31, path_lengths=0.379, generator_loss=2.03, critic_loss=1.42

In [ ]:
skorez = gan.critic_net(imz, F.one_hot(lbz, 2))

In [ ]:
jopa = torch.mean(skorez)

In [ ]:
jopa.backward()

In [ ]:
skorez.grad

In [ ]:
gan.

In [ ]:
a = gan(8, F.one_hot(torch.randint(0, 2, (8,)), 2).float())

In [ ]:
jopel = torch.mean(a[0])

In [ ]:
jopel.backward()

In [ ]:
jopel.grad